In [1]:
import os
import numpy as np
from PIL import Image
from storyart.novel2img import Novel2Img

/home/bitzerxiao/miniforge3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("tmp/inputs/test.txt", "r") as f:
    lines = f.readlines()
f.close()
history = list(map(lambda x: x.strip(), lines))
line = history[-1]

In [3]:
img_model_path = "/dataX/yiyong/checkpoints/FLUX.1-dev"
text_model_path = "/data1/yiyong/checkpoints/Qwen2.5-3B-Instruct"
config_path = "/home/bitzerxiao/project/StoryArt/storyart/configs"
model = Novel2Img(img_path=img_model_path, img_device="cuda:0", text_device="cuda:0", text_path=text_model_path, config_path=config_path, lowvarm=True)

[2024-12-02 16:41:02] start init


Loading pipeline components...: 100%|██████████| 7/7 [00:15<00:00,  2.26s/it]


[2024-12-02 16:41:35] finished load img_model


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.68it/s]


[2024-12-02 16:41:39] finished load text_model


In [4]:
info = model.info_extract(line=history[-2], novel="\n".join(history[:-1]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[2024-12-02 16:41:40] info extract: {"character": "苏暖暖", "scene": "杂志"}


In [5]:
describe = model.word2describe(info, line, context="现代", novel="\n".join(history))

[2024-12-02 16:41:42] character setting: {"sex": "young woman", "hair": "brown wavy hair", "clothes": "blue jeans and a red sweater"}
[2024-12-02 16:41:42] sucess setting 苏暖暖 -> 人,young woman,,brown wavy hair,blue jeans and a red sweater,solo,masterpiece,best quality,very aesthetic,absurdres
[2024-12-02 16:41:43] scene setting: {"image_prompt":"magazine on a wooden table with glossy images and smooth paper"} # english
[2024-12-02 16:41:43] error scene predict
[2024-12-02 16:41:44] pose predict: {"answer": "dance"}


In [6]:
model.name_to_sex = {}

In [ ]:
width, height = 832, 1216
batch_size = 2
imgs = model.describe2imgs(describe=describe, batch_size=batch_size, height=height, width=width, steps=20)

result_image = Image.new('RGB', (width*batch_size, height)) 

y = 0
x = 0
for i in range(batch_size):
    image = Image.fromarray(imgs[i])
    result_image.paste(image.resize((width,height)),(x,y))
    x += width
result_image